In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from glob import glob
import datetime
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [14]:
df=pd.read_csv('CoinText_final.csv')
df=df.drop('Unnamed: 0',axis=1)
df.head()

,CoinText,CoinDate
0,Crypto asset manager Valkyrie Investments want...,2022/12/30
1,The mining industry started 2022 off strong wi...,2022/12/30
2,Investment giant BlackRock (BLK) has committed...,2022/12/30
3,Noted growth investor Cathie Wood's ARK Invest...,2022/12/30
4,Good morning. Here’s what’s happening:Prices: ...,2022/12/30


"Last Tuesday, bitcoin (BTC) hit an all-time high measured in rubles (RUB). Does that really mean anything?Price feeds don’t tell the full story when comparing asset prices across different currencies, especially when one of those currencies is issued by a country facing financial sanctions for invading Ukraine (and the other is magic internet money).A quick look at USD/RUB tells most of the story. One U.S. dollar was worth 84 rubles on Friday, Feb. 25 and over 114 rubles on Tuesday, March 1. With that type of devaluation, you might reasonably expect many assets priced in rubles to hit all-time highs soon, especially if the trend continues.However, the ruble losing 37% of its U.S. dollar value would explain something around a 37% gain in “bitcoin in ruble” (USD and its stablecoin equivalents represent the most used trading pair for bitcoin, so this is an OK assumption). It wouldn’t immediately explain why bitcoin gained more than 55% (from 3.2 million to 4.9 million rubles); the excess

In [18]:
# 載入模型和tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# 設定要分類的標籤
labels = ['positive', 'negative', 'neutral']


# 將文本轉換為tokens並加入特殊tokens
inputs = tokenizer(str(df['CoinText'][96]), return_tensors='pt', padding=True, truncation=True)

# 進行預測
outputs = model(**inputs)
predicted_scores = torch.softmax(outputs.logits, dim=1)

# 挑選預測中得分最高的情緒類別
max_score_idx = torch.argmax(predicted_scores)
max_score_label = labels[max_score_idx]
max_score = predicted_scores[0][max_score_idx]
print(f'{max_score_label} is {max_score}')

neutral is 0.6768796443939209
